In [1]:
import numpy as np
import pandas as pd
import pyodbc as pyo
import sqlalchemy as sa
import numpy as np
import math
import datetime
import datetime as dt
#Kết nối SV_UAT (SQL_SV64)
connection_uri_UAT = (
    "mssql+pyodbc://PHLHCMHOPPBI02\PBIRS/PowerBI?driver=ODBC+Driver+17+for+SQL+Server"
)
conn = sa.create_engine(connection_uri_UAT)

<h3>1. Lấy bảng:

In [2]:
cutoff = input("Nhập kỳ CUTOFF cuối tháng YYYYMMDD: ")

Nhập kỳ CUTOFF cuối tháng YYYYMMDD: 20240126


<h5>1.1 Lấy bảng PY2

In [6]:
py2_query = f"""\n
SELECT * FROM DP_K2_CUTOFF
WHERE CUTOFF = {cutoff}
"""
py2_table = pd.read_sql(py2_query, conn)

<h5>1.2 Lấy bảng Commission

In [4]:
com_query = f"""\n
SELECT * FROM DP_TAGENTCOMM_TEST_CUTOFF
WHERE CUTOFF = {cutoff}
"""
com_table = pd.read_sql(com_query, conn)

<h3>2. Tính PY2

In [12]:
py2_t = py2_table
py2_t = py2_t.groupby(['AGENT NO'])[['Y2 ACTUAL PREM', 'Y2 EXPECTED PREM']].sum().reset_index()
py2_t['PY2'] = np.where(py2_t['Y2 EXPECTED PREM'] ==0, 0,
            round(py2_t['Y2 ACTUAL PREM']/py2_t['Y2 EXPECTED PREM'] , 4))
py2_t['PY2'] = np.where(py2_t['PY2'] >1, 1, py2_t['PY2'])
py2_t

,AGENT NO,Y2 ACTUAL PREM,Y2 EXPECTED PREM,PY2
0,60000481,16198000.0,1.619800e+07,1.0000
1,60000482,194406000.0,2.731560e+08,0.7117
2,60000717,106947000.0,1.936480e+08,0.5523
3,60000727,6480000.0,2.280500e+07,0.2841
4,60000823,60741000.0,1.362320e+08,0.4459
...,...,...,...,...
4701,69999895,0.0,1.407720e+08,0.0000
4702,69999896,0.0,9.939000e+06,0.0000
4703,69999899,71142000.0,2.469480e+08,0.2881
4704,69999997,81877000.0,1.203770e+08,0.6802


<h3>3. Tính Commission

In [18]:
com_t = com_table
com_t['Date_On_Set'] = pd.to_datetime(com_t['CUTOFF'].str[:4]+"-"+com_t['CUTOFF'].str[4:6]+"-"+com_t['CUTOFF'].str[6:])
com_t

,Policy No,Product Code,Premium transaction,Premium transaction name,Premium Collected,Collected Date,Applied Premium Date,FYC,RYC,Shared Commission Rate,...,Commission Agent,Transfer Date,Proposal Receive Date,Freelook,RISK_COMMENCE_DATE,Receive Policy date,Premium Year,CUTOFF,RETURN_DATE,Date_On_Set
0,80000013,EWR1,T642,Contract Issue,40000.0,2014-03-31,2014-03-31,8000.0,0.0,100.0,...,60000005,2014-03-31,2014-03-29,3581.0,2014-03-28,2014-04-08,1,20240126,None,2024-01-26
1,80000013,EWR1,B522,Premium Collection,40000.0,2014-07-29,2014-07-29,9000.0,0.0,100.0,...,60000005,2014-03-31,2014-03-29,3581.0,2014-03-28,2014-04-08,1,20240126,None,2024-01-26
2,80000013,ETR1,B522,Premium Collection,191000.0,2014-07-29,2014-07-29,40000.0,0.0,100.0,...,60000005,2014-03-31,2014-03-29,3581.0,2014-03-28,2014-04-08,1,20240126,None,2024-01-26
3,80000013,EE01,B522,Premium Collection,3640000.0,2014-07-29,2014-07-29,765000.0,0.0,100.0,...,60000005,2014-03-31,2014-03-29,3581.0,2014-03-28,2014-04-08,1,20240126,None,2024-01-26
4,80000013,ETR1,B522,Premium Collection,191000.0,2014-10-27,2014-10-27,40000.0,0.0,100.0,...,60000005,2014-03-31,2014-03-29,3581.0,2014-03-28,2014-04-08,1,20240126,None,2024-01-26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1589562,80148564,RHB1,T642,Contract Issue,1200000.0,2023-06-19,2023-06-19,180000.0,0.0,100.0,...,60079337,2023-06-19,2023-06-16,215.0,2023-06-16,2023-06-26,1,20240126,None,2024-01-26
1589563,80148564,URC5,T642,Contract Issue,720000.0,2023-06-19,2023-06-19,252000.0,0.0,100.0,...,60079337,2023-06-19,2023-06-16,215.0,2023-06-16,2023-06-26,1,20240126,None,2024-01-26
1589564,80148564,URA5,T642,Contract Issue,358000.0,2023-06-19,2023-06-19,125000.0,0.0,100.0,...,60079337,2023-06-19,2023-06-16,215.0,2023-06-16,2023-06-26,1,20240126,None,2024-01-26
1589565,80148565,UL05,T642,Contract Issue,34784000.0,2023-06-19,2023-06-19,13218000.0,0.0,100.0,...,60066942,2023-06-19,2023-06-17,219.0,2023-06-17,2023-06-22,1,20240126,None,2024-01-26


In [19]:
if ('20230531' in com_t['CUTOFF'].values):
    com_t.loc[:,'Start_Month'] = '2023-04-30'
elif ('20220228' in com_t['CUTOFF'].values):
    com_t.loc[:,'Start_Month'] = '2022-01-28'
elif ('20220429' in com_t['CUTOFF'].values):
    com_t.loc[:,'Start_Month'] = '2022-03-31'
elif ('20220531' in com_t['CUTOFF'].values):
    com_t.loc[:,'Start_Month'] = '2022-04-30' 
elif ('20240229' in com_t['CUTOFF'].values):
    com_t.loc[:,'Start_Month'] = '2024-01-27'
else: 
    com_t['Start_Month'] = (com_t['Date_On_Set'].dt.floor('d') + pd.offsets.MonthEnd(0) - pd.offsets.MonthBegin(1))
com_t

,Policy No,Product Code,Premium transaction,Premium transaction name,Premium Collected,Collected Date,Applied Premium Date,FYC,RYC,Shared Commission Rate,...,Transfer Date,Proposal Receive Date,Freelook,RISK_COMMENCE_DATE,Receive Policy date,Premium Year,CUTOFF,RETURN_DATE,Date_On_Set,Start_Month
0,80000013,EWR1,T642,Contract Issue,40000.0,2014-03-31,2014-03-31,8000.0,0.0,100.0,...,2014-03-31,2014-03-29,3581.0,2014-03-28,2014-04-08,1,20240126,None,2024-01-26,2024-01-01
1,80000013,EWR1,B522,Premium Collection,40000.0,2014-07-29,2014-07-29,9000.0,0.0,100.0,...,2014-03-31,2014-03-29,3581.0,2014-03-28,2014-04-08,1,20240126,None,2024-01-26,2024-01-01
2,80000013,ETR1,B522,Premium Collection,191000.0,2014-07-29,2014-07-29,40000.0,0.0,100.0,...,2014-03-31,2014-03-29,3581.0,2014-03-28,2014-04-08,1,20240126,None,2024-01-26,2024-01-01
3,80000013,EE01,B522,Premium Collection,3640000.0,2014-07-29,2014-07-29,765000.0,0.0,100.0,...,2014-03-31,2014-03-29,3581.0,2014-03-28,2014-04-08,1,20240126,None,2024-01-26,2024-01-01
4,80000013,ETR1,B522,Premium Collection,191000.0,2014-10-27,2014-10-27,40000.0,0.0,100.0,...,2014-03-31,2014-03-29,3581.0,2014-03-28,2014-04-08,1,20240126,None,2024-01-26,2024-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1589562,80148564,RHB1,T642,Contract Issue,1200000.0,2023-06-19,2023-06-19,180000.0,0.0,100.0,...,2023-06-19,2023-06-16,215.0,2023-06-16,2023-06-26,1,20240126,None,2024-01-26,2024-01-01
1589563,80148564,URC5,T642,Contract Issue,720000.0,2023-06-19,2023-06-19,252000.0,0.0,100.0,...,2023-06-19,2023-06-16,215.0,2023-06-16,2023-06-26,1,20240126,None,2024-01-26,2024-01-01
1589564,80148564,URA5,T642,Contract Issue,358000.0,2023-06-19,2023-06-19,125000.0,0.0,100.0,...,2023-06-19,2023-06-16,215.0,2023-06-16,2023-06-26,1,20240126,None,2024-01-26,2024-01-01
1589565,80148565,UL05,T642,Contract Issue,34784000.0,2023-06-19,2023-06-19,13218000.0,0.0,100.0,...,2023-06-19,2023-06-17,219.0,2023-06-17,2023-06-22,1,20240126,None,2024-01-26,2024-01-01


In [23]:
com_t = com_t[(com_t['Applied Premium Date'] >= com_t['Start_Month']) & (com_t['Applied Premium Date'] <= com_t['Date_On_Set'])]
com_t = com_t[~com_t['Policy Status'].isin(['CF', 'DC', 'FL', 'NT', 'WD', 'PO', 'SU', 'TR'])]
com_t

,Policy No,Product Code,Premium transaction,Premium transaction name,Premium Collected,Collected Date,Applied Premium Date,FYC,RYC,Shared Commission Rate,...,Transfer Date,Proposal Receive Date,Freelook,RISK_COMMENCE_DATE,Receive Policy date,Premium Year,CUTOFF,RETURN_DATE,Date_On_Set,Start_Month
372,80000483,RAC2,B522,Premium Collection,286000.0,2024-01-16,2024-01-16,0.0,9000.0,100.0,...,2016-03-11,2015-07-17,3112.0,2015-07-16,2015-07-21,9,20240126,None,2024-01-26,2024-01-01
373,80000483,EE01,B522,Premium Collection,4411000.0,2024-01-16,2024-01-16,0.0,88000.0,100.0,...,2016-03-11,2015-07-17,3112.0,2015-07-16,2015-07-21,9,20240126,None,2024-01-26,2024-01-01
374,80000483,EWR1,B522,Premium Collection,88000.0,2024-01-16,2024-01-16,0.0,2000.0,100.0,...,2016-03-11,2015-07-17,3112.0,2015-07-16,2015-07-21,9,20240126,None,2024-01-26,2024-01-01
375,80000483,RAC2,B522,Premium Collection,286000.0,2024-01-16,2024-01-16,0.0,9000.0,100.0,...,2016-03-11,2015-07-17,3112.0,2015-07-16,2015-07-21,9,20240126,None,2024-01-26,2024-01-01
376,80000483,ETR1,B522,Premium Collection,629000.0,2024-01-16,2024-01-16,0.0,13000.0,100.0,...,2016-03-11,2015-07-17,3112.0,2015-07-16,2015-07-21,9,20240126,None,2024-01-26,2024-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1571737,80131849,UL05,TU79,IAV Top-up(2021),1730000.0,2024-01-16,2024-01-16,0.0,18000.0,100.0,...,2023-01-11,2023-01-11,343.0,2023-01-11,2023-02-18,2,20240126,None,2024-01-26,2024-01-01
1571744,80131851,URC5,B522,Premium Collection,509000.0,2024-01-22,2024-01-22,0.0,61000.0,100.0,...,2023-01-11,2023-01-11,348.0,2023-01-11,2023-02-13,2,20240126,None,2024-01-26,2024-01-01
1571745,80131851,URA5,B522,Premium Collection,90000.0,2024-01-22,2024-01-22,0.0,6000.0,100.0,...,2023-01-11,2023-01-11,348.0,2023-01-11,2023-02-13,2,20240126,None,2024-01-26,2024-01-01
1571746,80131851,UL05,B522,Premium Collection,3676000.0,2024-01-22,2024-01-22,0.0,368000.0,100.0,...,2023-01-11,2023-01-11,348.0,2023-01-11,2023-02-13,2,20240126,None,2024-01-26,2024-01-01


<h5>3.1 Tính groupby Com

In [27]:
RYC_com = com_t
RYC_com = RYC_com.groupby(['Commission Agent'])['RYC'].sum().reset_index()
RYC_com

,Commission Agent,RYC
0,60000194,338000.0
1,60000288,362000.0
2,60000449,70000.0
3,60000450,2589000.0
4,60000477,60000.0
...,...,...
1572,60079599,842000.0
1573,60080684,0.0
1574,60080713,553000.0
1575,60082246,0.0


<h3> 4. Tính PY2 Bonus

In [36]:
bonus_py2 = pd.merge(py2_t, RYC_com, left_on = 'AGENT NO', right_on = 'Commission Agent', how ='left')
bonus_py2 = bonus_py2[['AGENT NO', 'PY2','RYC']]
bonus_py2['%Bonus'] = np.where((bonus_py2['PY2'] >= 0.7) & (bonus_py2['PY2'] < 0.8), 0.3, 
                    np.where((bonus_py2['PY2'] >= 0.8) & (bonus_py2['PY2'] < 0.9), 0.35,
                    np.where((bonus_py2['PY2'] >= 0.9), 0.5,0)))
bonus_py2['Bonus RYC'] = bonus_py2['RYC'] * bonus_py2['%Bonus']
bonus_py2.sort_values('Bonus RYC', ascending = False)

,AGENT NO,PY2,RYC,%Bonus,Bonus RYC
1103,60045133,0.7895,49122000.0,0.30,14736600.0
732,60036079,1.0000,13615000.0,0.50,6807500.0
211,60020075,1.0000,13122000.0,0.50,6561000.0
1983,60056020,1.0000,8186000.0,0.50,4093000.0
136,60014571,0.8321,7425000.0,0.35,2598750.0
...,...,...,...,...,...
4701,69999895,0.0000,NaN,0.00,NaN
4702,69999896,0.0000,NaN,0.00,NaN
4703,69999899,0.2881,NaN,0.00,NaN
4704,69999997,0.6802,NaN,0.00,NaN


In [37]:
bonus_py2.to_csv('bonus_py2.csv')